In [8]:
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras import datasets, layers, models
import tensorflow as tf

import joblib


In [9]:
#defining some parameters
batch_size = 32
img_height = 128
img_width = 128

In [10]:
#gathering the data and separating into training and validation datasets
data_dir = 'data/images/archive/'
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.3,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)



Found 20000 files belonging to 2 classes.
Using 14000 files for training.


In [11]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.3,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)


Found 20000 files belonging to 2 classes.
Using 6000 files for validation.


In [12]:
#The features
class_names = train_ds.class_names
print(class_names)


['with_mask', 'without_mask']


In [ ]:
#Designing the CNN
model = models.Sequential()
model.add(layers.Conv2D(128, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(2,activation = 'softmax'))

In [ ]:
#Compiling the model
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),
  metrics=['accuracy'])




In [ ]:
#Training the model
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=1, 
  shuffle=True
)


In [1]:
#used to load images and plot them
import cv2 as cv

In [2]:
#Function which when given an image from the dataset 
#it outputs the image and the prediction for the image
def predictions(img="with_mask/with-mask-default-mask-seed5777.png"):
    img_path = "data/images/archive/" + img
    image1 = cv.imread(img_path)
    image1 = cv.cvtColor(image1, cv.COLOR_BGR2RGB )
    image1= cv.resize(image1, (img_height,img_width), interpolation = cv.INTER_AREA)
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(img_height, img_width))
    plt.imshow(image1)
    plt.show()
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_batch = np.expand_dims(img_array, axis=0)
    tf.image.rgb_to_grayscale(img_batch)
    prediction = model.predict(img_batch)
    print(class_names[np.argmax(prediction)])
    print(prediction[0])


In [3]:
#Testing the model
predictions("with_mask/with-mask-default-mask-seed2322.png")


NameError: name 'img_height' is not defined

In [ ]:
#Saving the model
filename = 'trained_mask_model.sav'
joblib.dump(model, filename)